In [ ]:
sc.list_packages()

In [ ]:
%info

In [ ]:
# {"conf":{"spark.pyspark.python": "python3.7",
#          "spark.pyspark.virtualenv.enabled": "true",
#          "spark.pyspark.virtualenv.type": "native",
#          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"}
# }

In [9]:
# sc.install_pypi_package("pandas==0.25.1")
# sc.install_pypi_package("requests")
# sc.install_pypi_package("scipy")
# sc.install_pypi_package("Pillow")
# sc.install_pypi_package("h5py==3.0.0")
# sc.install_pypi_package("s3fs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Import needed library

In [149]:
import pandas as pd
import requests
from io import StringIO  # python3 (or BytesIO for python2)
import io
import tensorflow as tf
import ast
from tensorflow.keras.applications import MobileNetV2
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.linalg import *
from  pyspark.mllib.linalg.distributed import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import monotonically_increasing_id
from functools import reduce
from pyspark.sql.window import *
import numpy as np
import os
from scipy.spatial.distance import cosine
from pyspark.sql.types import *
import requests
import boto3
from PIL import Image

# tf.enable_eager_execution()

np.random.seed(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Get the origin dataframe 

In [150]:
client = boto3.client('s3')

def remove(link):
    new_link = link.split("/")
    new_link = new_link[:-1]
    string = '/'.join(new_link)
    return string

def removeDuplicate(test_list):
    res = []
    for i in test_list:
        if i not in res:
            res.append(i)
    return res
    

def createDataFrame():
    
    sub_list_1 = []

    bucket = client.list_objects_v2(Bucket="kinesis-s3records-16jckkpyv314r", Prefix ='firehose')
    for content in bucket["Contents"]:
        key = content["Key"]
        sub_list_1.append(remove(key))
        
    sub_list_1 = removeDuplicate(sub_list_1)
        
    
    sub_list_2 = []
    
    
    for i in range(len(sub_list_1)):
        sub_list_2.append("s3://kinesis-s3records-16jckkpyv314r/" + sub_list_1[i]+"/")
    

    # Creating the dataframe and append list of object to it
    schema = StructType([
        StructField('caption', StringType(), True),
        StructField('hashtag', StringType(), True),
        StructField('image_url', StringType(), True)
    ])
    df = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)
    for s3 in sub_list_2:
        df_added = spark.read.json(s3)
        df = df.union(df_added)

    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [151]:
df = createDataFrame()
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|             caption|             hashtag|           image_url|
+--------------------+--------------------+--------------------+
|Read caption 📌

...|["btsatunga", "un...|https://scontent-...|
|The whole event o...|["unitednations",...|https://scontent-...|
|New AirPods Pro a...|["apple", "global...|https://scontent-...|
|Ko Phyu Gyi, a vi...|["Letkoakpin vill...|https://scontent-...|
|Get your hands on...|["peing", "nft", ...|https://scontent-...|
|....................|["btsquotes", "un...|https://scontent-...|
|https://kaderjila...|["HappySunshineDa...|https://scontent-...|
|Where's the lie t...|["bts", "btsedits...|https://scontent-...|
|   #godmorningmonday|["godmorningmonday"]|https://scontent-...|
|Have you register...|["unga", "unwomen...|https://scontent-...|
|New AirPods Pro a...|["apple", "global...|https://scontent-...|
|New AirPods Pro a...|["apple", "global...|https://scontent-...|
|New AirPods Pro a...|["ap

In [152]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

421

# Data engineer for the original datasets

### 1.0 Create hashtags features

In [153]:
# get the hashtags from caption
import re
def find_hashtag(caption):
    to_replace = ["\n", ".", ","]
    temp = caption
    for i in to_replace:
        temp = temp.replace(i,"")
    hashtags = re.findall("#[A-Za-z]+", temp)
    return hashtags    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [154]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- caption: string (nullable = true)
 |-- hashtag: string (nullable = true)
 |-- image_url: string (nullable = true)

In [155]:
# get the pandas version for creating img and hashtags datasets
df_pd = df.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [156]:
df_pd.caption[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Read caption 📌\n\nThese ANGELS are constantly creating history, "and there\'s no looking back"💪\n.\nGoshhh its day 3 and I can\'t stop screaming about how much proud I am for our boys😭💜\n.\nThey were singing at dancing at UN headquarters , mann its not joke anymore \nI proudly say this "YES THEY ARE THE KINGS OF THIS ERA"\n.\n.\nPLEASE DON\'T REPOST WITHOUT CREDITS 💮\n.\n\n#btsatunga #unga #btsaspresidentenvoy #dramaqueen #BTS #taehyung\n#v #namjoon #kimnamjoon #kimnamjoonkimseokjinminyoongijunghoseokparkjiminkimtaehyungjeonjungkookbts #jimin #jungkook #yoongi #hobi #hoseak #jin #seokjin #kpop #army #btsarmy #btsarmyforever #butter #btsbutter #hybe #hybelabels #buttermv #bangtanboys #btsedit #btslove #purpleyou💜'

In [157]:
df_pd["hashtags"] = df_pd.caption.apply(lambda x: find_hashtag(x))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [158]:
df_pd["hashtags"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0      [#btsatunga, #unga, #btsaspresidentenvoy, #dra...
1      [#unitednations, #unga, #bts, #bangtanboys, #b...
2      [#apple, #globalarmsembargo, #unga, #godmornin...
3      [#Letkoakpin, #Natmauk, #globalarmsembargo, #s...
4      [#peing, #nft, #emmys, #ps, #amici, #nowplayin...
                             ...                        
416    [#happyseungminday, #happybirthdayseungmin, #s...
417    [#NFTcollectibles, #nftartgallery, #nftcollect...
418              [#seungminstraykids, #seungminbirthday]
419    [#Original, #song, #straykids, #straykidsseung...
420                  [#jinyoung, #allrounderjinyoungday]
Name: hashtags, Length: 421, dtype: object

### 2.0 Extract image deep features

In [91]:
# function to load image as tensor from url 
def get_image(url):
    r = requests.get(url)
    img = tf.io.decode_image(r.content)
    return img 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [159]:
# Construct neural net to extract image features 
def construct_neural_network():
    img_shape = (160, 160, 3)

    # Create the base model from the pre-trained model MobileNet V2
    base_model = MobileNetV2(input_shape=img_shape, include_top=False, weights='imagenet')

    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

    neural_network = tf.keras.Sequential([
      base_model,
      global_average_layer,
    ])
    
    return neural_network

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [160]:
# extract features get the deep features from neural network to use later 
def extract_features(image, neural_network):
    """Return a vector of 1280 deep features for image."""
    images = np.expand_dims(image, axis=0)
    deep_features = neural_network.predict(images)[0]
    return deep_features

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [161]:
# transform image to numpy arrays
def prepare_image(img,height = 160,width = 160):
    img = tf.cast(img, tf.float32)
    img = (img/127.5) - 1
    img = tf.image.resize(img, (height, width))
    # Reshape grayscale images to match dimensions of color images
    if img.shape != (160, 160, 3):
        img = tf.concat([img, img, img], axis=2)
    return img.numpy()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [162]:
# use for extract deep features directly from url
def get_vector_url(url,neural_network):
    image = prepare_image(get_image(url))
    return extract_features(image,neural_network)

# use for EMR notebook 
def get_vector_img(img,neural_network):
    img = prepare_image(img)
    return extract_features(img,neural_network)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [163]:
df_pd["img_name"] = df_pd.index.to_series().apply(lambda x: f"img_{str(x)}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [164]:
neural_net = construct_neural_network()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [165]:
def get_deep_feature(url,neural_net):
    try:
        return get_vector_url(url,neural_net)
    except:
        pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [166]:
# funtion to loading the deep features from saved dataframe
def create_deepfeatures(index,original_dataframe,saved_dataframe,neural_net):
    if index <= saved_dataframe.index[-1]:
        return saved_dataframe.loc[index,"deep_feature"]
    else:
        try:
            url = original_dataframe.loc[index,"image_url"]
            return get_deep_feature(url,neural_net)
        except:
            return np.zeros(1280)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [167]:
# function to read csv file from bucket
def read_csv_file(bucket,key):
    obj = client.get_object(Bucket=bucket, Key=f'dataframe/{key}')
    df = pd.read_csv(io.BytesIO(obj['Body'].read()))
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [168]:
# function to write csv file to bucket
def save_df(df,file_name):
    bucket = "kinesis-s3records-16jckkpyv314r"
    csv_buffer = StringIO()
    df.to_csv(csv_buffer)

    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket, f'dataframe/{file_name}').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [169]:
# load saved deep_features 
bucketname = "kinesis-s3records-16jckkpyv314r"
keyname = "df_deep_features.csv"
saved_df_features = read_csv_file(bucketname,keyname)

saved_df_features["deep_feature"] = saved_df_features["deep_feature"].apply(lambda x: np.fromstring(x[1:-1],sep=", ") )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [170]:
saved_df_features

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     Unnamed: 0  ...                                       deep_feature
0             0  ...  [0.0, 0.0, 0.865784764289856, 0.00069944956339...
1             1  ...  [0.879599928855896, 0.23585304617881775, 1.039...
2             2  ...  [0.0, 0.0, 0.7003890872001648, 0.1560206115245...
3             3  ...  [0.0075896806083619595, 0.0, 0.532040476799011...
4             4  ...  [0.0, 0.11799856275320053, 0.0, 0.088474199175...
..          ...  ...                                                ...
416         416  ...  [0.06956370174884796, 0.039646413177251816, 0....
417         417  ...  [3.126276969909668, 0.0, 0.0, 0.0, 0.209734916...
418         418  ...  [0.15421395003795624, 0.655612051486969, 0.333...
419         419  ...  [0.2562243640422821, 0.0, 0.0, 2.1857922077178...
420         420  ...  [0.0, 0.0, 0.0, 3.2210769653320312, 1.97364747...

[421 rows x 3 columns]

In [172]:
df_pd["deep_feature"] = df_pd.index.to_series().apply(lambda x:  create_deepfeatures(x, df_pd,saved_df_features ,neural_net))
# df_pd["deep_feature"] = df_pd.image_url.apply(lambda x: get_vector_url(x,neural_net))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [173]:
df_pd["deep_feature"] 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0      [0.0, 0.0, 0.865784764289856, 0.00069944956339...
1      [0.879599928855896, 0.23585304617881775, 1.039...
2      [0.0, 0.0, 0.7003890872001648, 0.1560206115245...
3      [0.0075896806083619595, 0.0, 0.532040476799011...
4      [0.0, 0.11799856275320053, 0.0, 0.088474199175...
                             ...                        
416    [0.06956370174884796, 0.039646413177251816, 0....
417    [3.126276969909668, 0.0, 0.0, 0.0, 0.209734916...
418    [0.15421395003795624, 0.655612051486969, 0.333...
419    [0.2562243640422821, 0.0, 0.0, 2.1857922077178...
420    [0.0, 0.0, 0.0, 3.2210769653320312, 1.97364747...
Name: deep_feature, Length: 421, dtype: object

In [174]:
df_proceed = df_pd[["hashtags","deep_feature"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [175]:
df_save = df_proceed.copy()
df_save["deep_feature"] = df_save.deep_feature.apply(lambda x: str(x.tolist()))
save_df(df_save,"df_deep_features.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 3.0 load addition dataset 

In [176]:
# load the additional data
df_add_Data = read_csv_file(bucketname,"df_add.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [177]:
df_add_Data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index(['Unnamed: 0', 'hashtags', 'deep_feature'], dtype='object')

In [178]:
df_add_Data.iloc[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Unnamed: 0                                                      0
hashtags        ['#travelcalifornia', '#traveling', '#travelli...
deep_feature    [0.7958510518074036, 1.4362449645996094, 0.0, ...
Name: 0, dtype: object

In [179]:
df_add_Data["deep_feature"] = df_add_Data.deep_feature.apply(lambda x: np.fromstring(x[1:-1],sep =","))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [180]:
df_add_Data["hashtags"] = df_add_Data.hashtags.apply(lambda x: ast.literal_eval(x))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [181]:
df_add_Data["deep_feature"][0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([0.79585105, 1.43624496, 0.        , ..., 0.86633819, 1.60301924,
       0.3259587 ])

In [182]:
df_add_Data["hashtags"][0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['#travelcalifornia', '#traveling', '#travellifecc', '#travels', '#traveladdict', '#traveler', '#travel', '#usatravel', '#destinations', '#californialife', '#instatravel', '#travelearth', '#usa', '#santamonica', '#californiatravel', '#explorecalifornia', '#travelgram', '#travelphotography', '#beautifulsunset', '#wonderfulsunset', '#wanderlust', '#photography', '#traveltheearth', '#santamonicabeach', '#travelearthu', '#travellife', '#travelblogger', '#santamonicaphotographer', '#traveltheworld', '#travelusa']

In [183]:
df_use = pd.concat([df_proceed,df_add_Data],ignore_index = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [184]:
for i in df_use.columns:
    if "Unnamed" in i:
        del df_use[i]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [185]:
df_use.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index(['deep_feature', 'hashtags'], dtype='object')

In [186]:
df_save = df_use.copy()
df_save["deep_feature"] = df_save.deep_feature.apply(lambda x: str(x.tolist()))
save_df(df_save,"df_features.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.0 Construct hashtags datasets for training als model 

In [187]:
# Create a flattened list of all hashtags
all_hashtags = [hashtag for hashtags in df_use['hashtags'] for hashtag in hashtags]

# Coerce to a set to remove duplicate entries
# Sort to ensure reproducibility of results
all_hashtags = sorted(list(set(all_hashtags)))

# Build lookup for finding hashtag number based on hashtag name
hashtag_lookup = {hashtag: i for i, hashtag in enumerate(all_hashtags)}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [188]:
# create dataframe to save all hashtags
hashtags_df = pd.DataFrame.from_dict(hashtag_lookup, orient='index')
hashtags_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

             0
#A           0
#ABURTOTEAM  1
#ADORABLE    2
#AFAA        3
#AMG         4

In [189]:
hashtags_df = hashtags_df.reset_index()
hashtags_df.columns = ['hashtag', 'id']
hashtags_df.index = hashtags_df['id']
hashtags_df.drop('id', axis=1, inplace=True)
hashtags_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

               hashtag
id                    
0                   #A
1          #ABURTOTEAM
2            #ADORABLE
3                #AFAA
4                 #AMG
...                ...
20409  #zwergkaninchen
20410      #zwergspitz
20411     #zwergwidder
20412          #zwierz
20413          #zyklus

[20414 rows x 1 columns]

In [190]:
# save the hashtags
save_df(hashtags_df,"hashtags.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [191]:
# construct dataframe for recsys of hashtag_id and image_id 
hashtag_rec_data = []
for i in df_use.index:
    hashtag_list = df_use.loc[i, 'hashtags']
    for hashtag in hashtag_list:
        hashtag_rec_data.append(
            {'image_id': i,
             'hashtag_id': hashtag_lookup[hashtag],
             'rating': 1}
        )
hashtag_rec_data_df = pd.DataFrame(hashtag_rec_data)
hashtag_rec_data_df.tail()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       image_id  hashtag_id  rating
64356      3411        3986       1
64357      3411         692       1
64358      3411       14763       1
64359      3411       14791       1
64360      3411        9507       1

In [192]:
# save the recommendation dataframe of hashtags
save_df(hashtag_rec_data_df,"rec_hashtags.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 5.0 Training the ALS model

In [125]:
als = ALS(userCol='image_id',
          itemCol='hashtag_id',
          implicitPrefs=True,
          alpha=40)

als.setSeed(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ALS_495840bb4a26

In [193]:
sqlcontext = SQLContext(sc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [194]:
rec_hashtags = sqlcontext.read.csv("s3://kinesis-s3records-16jckkpyv314r/dataframe/rec_hashtags.csv",header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [195]:
rec_hashtags.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------+----------+------+
|_c0|image_id|hashtag_id|rating|
+---+--------+----------+------+
|  0|       0|      3885|     1|
|  1|       0|     18975|     1|
|  2|       0|      3883|     1|
|  3|       0|      6411|     1|
|  4|       0|        76|     1|
|  5|       0|     17922|     1|
|  6|       0|     19082|     1|
|  7|       0|     13294|     1|
|  8|       0|     10816|     1|
|  9|       0|     10817|     1|
| 10|       0|     10478|     1|
| 11|       0|     10537|     1|
| 12|       0|     20287|     1|
| 13|       0|      9307|     1|
| 14|       0|      9433|     1|
| 15|       0|     10481|     1|
| 16|       0|     16645|     1|
| 17|       0|     10952|     1|
| 18|       0|      2101|     1|
| 19|       0|      3879|     1|
+---+--------+----------+------+
only showing top 20 rows

In [196]:
rec_hashtags.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

64361

In [197]:
rec_hashtags.toPandas().tail(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         _c0 image_id hashtag_id rating
64357  64357     3411        692      1
64358  64358     3411      14763      1
64359  64359     3411      14791      1
64360  64360     3411       9507      1

In [198]:
rec_hashtags.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: string (nullable = true)
 |-- image_id: string (nullable = true)
 |-- hashtag_id: string (nullable = true)
 |-- rating: string (nullable = true)

In [199]:
# cast the features to the right data type
for i in ["image_id","hashtag_id","rating"]:
    rec_hashtags = rec_hashtags.withColumn(i, rec_hashtags[i].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [200]:
# fit the model
als_model = als.fit(rec_hashtags)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [201]:
# get image and hashtags features extracted from als
img_features = als_model.userFactors.toPandas()
hashtag_features = als_model.itemFactors.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [202]:
# set id to become the features dataframes index
img_features = img_features.set_index("id")
hashtag_features = hashtag_features.set_index("id")

# save the features
img_features_save = img_features.copy()
hashtag_features_save = hashtag_features.copy()
img_features_save["features"] = img_features_save["features"].apply(lambda x:str(x))
hashtag_features_save["features"] = hashtag_features_save["features"].apply(lambda x:str(x))

# save features for later used 
save_df(img_features_save,"img_features.csv")
save_df(hashtag_features_save,"hashtag_features.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# 6.0 function to predict the hashtags

In [203]:
# helper function for image hashtag generators
def find_neighbor_vectors(features,recommender_df, k=5):
    """Find image features (user vectors) for similar images."""
    rdf = recommender_df.copy()
    rdf['dist'] = rdf['deep_feature'].apply(lambda x: cosine(x, features))
    rdf = rdf.sort_values(by='dist')
    return rdf.head(k)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [204]:
# helper function to load image from s3 
def fetch_image_from_s3_to_array(client,bucket, key):
    """Fetches an image from S3 and returns a numpy array."""
    response = client.get_object(Bucket=bucket, Key=key)
    body = response["Body"]
    data = body.read()
    f = BytesIO(data)
    image = Image.open(f)
    image_data = np.asarray(image)
    return image_data
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [205]:
def get_hashtags(df_features, hashtags_df,images_features,hashtag_features,neural_net = neural_net, url = None,client = client ,bucket=None,key = None,number = 20):
    # case url, it is a future feature as well as a tool for fast evaluate and debug
    if url:
        deep_features = get_vector_url(url,neural_net)
    else:
        # case load data from s3
        image = fetch_image_from_s3_to_array(client,bucket,key)
        deep_features = get_vector_img(image,neural_net)
    
    # get the id of images that share the common deep features with target image
    df_rec = df_features.copy()
    chosen = find_neighbor_vectors(deep_features,df_rec)
    
    # helper function to filter out image that having no hashtags in original dataset
    def get_list(chosen_vectors,images_features):
        list_return = []
        for i in chosen_vectors.index.tolist():
            try:
                list_return.append(images_features.loc[i,"features"])
            except:
                pass
        return list_return
    
    # calculate the average als image factors of target image
    list_features_combine = get_list(chosen,images_features )
    average_features = np.asarray(list_features_combine).mean(axis = 0)
    
    # perform dot product and sorted hashtags based on the dot product
    hashtags = hashtag_features.copy()
    hashtags["dot_product"] = hashtags.features.apply(lambda x: np.asarray(x).dot(average_features))
    final_recs = hashtags.sort_values(by='dot_product', ascending=False).head(number)
    # Look up hashtags by their numeric IDs
    output = []
    for hashtag_id in final_recs.index.values:
        output.append(hashtags_df.iloc[hashtag_id]['hashtag'])

    return output

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [206]:
# Demo using url case
url = "https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/dog-puppy-on-garden-royalty-free-image-1586966191.jpg?crop=1.00xw:0.669xh;0,0.190xh&resize=1200:*"
url2 = "https://storage.googleapis.com/cdn.nhanh.vn/store2/69685/bn/sb_1585213942_796.jpg"
url3 = "https://sansangduhoc.vn/wp-content/uploads/2020/11/17.11-travel-1.jpg"
hashtags_target = get_hashtags(df_use,hashtags_df,img_features,hashtag_features,url =url )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [207]:
hashtags_target

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['#animals', '#animal', '#cat', '#pets', '#cats', '#cute', '#pet', '#love', '#dog', '#instagram', '#dogs', '#nature', '#petsofinstagram', '#bhfyp', '#of', '#petstagram', '#animallovers', '#catsofinstagram', '#puppy', '#dogsofinstagram']

# 7.0 demo of loading data for production 

In [141]:
# load and preprocess saved image deep featured dataset
df_features = read_csv_file(bucketname,"df_features.csv")
df_features.columns 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index(['Unnamed: 0', 'deep_feature', 'hashtags'], dtype='object')

In [142]:
df_features["deep_feature"] = df_features["deep_feature"].apply(lambda x: np.fromstring(x[1:-2],sep=", "))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [143]:
del df_features["Unnamed: 0"]
df_features

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                           deep_feature                                           hashtags
0     [0.0, 0.0, 0.865784764289856, 0.00069944956339...  ['#btsatunga', '#unga', '#btsaspresidentenvoy'...
1     [0.879599928855896, 0.23585304617881775, 1.039...  ['#unitednations', '#unga', '#bts', '#bangtanb...
2     [0.0, 0.0, 0.7003890872001648, 0.1560206115245...  ['#apple', '#globalarmsembargo', '#unga', '#go...
3     [0.0075896806083619595, 0.0, 0.532040476799011...  ['#Letkoakpin', '#Natmauk', '#globalarmsembarg...
4     [0.0, 0.11799856275320053, 0.0, 0.088474199175...  ['#peing', '#nft', '#emmys', '#ps', '#amici', ...
...                                                 ...                                                ...
3407  [1.8191308975219727, 0.6857613921165466, 0.283...  ['#buildings', '#building', '#home', '#ezstudi...
3408  [1.3507258892059326, 1.7377569675445557, 0.992...  ['#buildings', '#be', '#architecturephotograph...
3409  [0.0, 0.0, 0.5229275226593018, 

In [144]:
# load the saved hashtags
hashtags_saved = read_csv_file(bucketname,"hashtags.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [145]:
# load the image factors of als models
image_features_saved = read_csv_file(bucketname,"img_features.csv")
image_features_saved["features"] = image_features_saved["features"].apply(lambda x: ast.literal_eval(x))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [146]:
# load the hashtags factors of als models
hashtag_features_saved = read_csv_file(bucketname,"hashtag_features.csv")
hashtag_features_saved["features"] = hashtag_features_saved["features"].apply(lambda x: ast.literal_eval(x))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [148]:
target_hashtags2 = get_hashtags(df_features,hashtags_saved,img_features,hashtag_features,url =url )
target_hashtags2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['#animals', '#animal', '#cat', '#pets', '#cats', '#cute', '#pet', '#love', '#dog', '#instagram', '#dogs', '#nature', '#petsofinstagram', '#bhfyp', '#of', '#petstagram', '#animallovers', '#catsofinstagram', '#puppy', '#dogsofinstagram']